## Univariate spatial prewhitening using ResMS.nii

This Jupyter Noteboox shows how to perform univariate spatial prewhitening of general linear model (GLM) coefficients (i.e., betas) using the residual mean squares (ResMS) output by the statistical parametric modelling (SPM) toolbox after GLM estimation. In multivariate fMRI analysis, this step allows to take into account that individual voxels are affected by different levels of noise (see Walther et al., 2016). 

In [49]:
# relevant imports
import numpy as np
import rsatoolbox as rsa
from scipy.linalg import sqrtm
import nibabel as nb
import numpy as np

np.random.seed(7)


Betas stored in .nii image files after SPM estimation can be loaded using nibabel.load() and then extracted as an i-by-j-by-k 3D tensor with the .get_fdata() method.

In [50]:
# betas = nb.load('path/to/glmDir/beta_0001.nii')

Alternatively, you can load betas from a specific region of interest (ROI) using world coordinates (x, y, z) with nitools.sample_image(). This may help you save space on disk. In this case, betas from the desired locations are loaded as a vector.

In [51]:
# betas = nt.sample_image('path/to/glmDir/beta_0001.nii', x, y, z)

Here, we will simulate a vector of betas similar to the output of nitools.sample_image().

In [47]:
betas = np.random.random(size=(1000,))

Univariate pre-whitening involves normalizing the activity estimated in each voxel by the standard deviation of its residuals. SPM stores the residual mean squares in the ResMS.nii file in the same directory as the beta_XXXX.nii files. You can load them using nibabel.load('path/to/glmDir/ResMS.nii').get_fdata() or nitools.sample_image(). Here, we will simulate a vector of mean squares as done for the betas.

In [48]:
# load residual mean square
ResMS = np.random.random(size=(1000,))

To pre-whitening our betas, we now need to divide our betas by the square root of ResMS.

In [41]:
betas_prewhitened = betas / np.sqrt(ResMS)

Calculating the euclidean distances between patterns of prewhitened betas is equal to computing the mahalanobis distance using the (diagonalized) inverse of ResMS instead of the covariance matrix. We can do this as a sanity check that the prewhitening was performed correctly. Let's first define a set of 5 betas each from a different condition.

In [42]:
betas = np.random.random(size=(5, 1000,))
betas_prewhitened = betas / np.sqrt(ResMS)
nCond = betas_prewhitened.shape[0]

We will now use the rsatoolbox to calculate the multivariate distances between conditions. First need to define a dataset object with the non-prewhitened and prewhitened betas

In [43]:
# create a dataset object
nVox = betas_prewhitened.shape[1]
des = {'session': 1, 'subj': 1}
obs_des = {'conds': np.array(['cond_%02d' % x for x in np.arange(nCond)])}
chn_des = {'voxels': np.array(['voxel_' + str(x) for x in np.arange(nVox)])}

# non-prewhitened dataset object
dataset = rsa.data.Dataset(measurements=betas,
                   descriptors=des,
                   obs_descriptors=obs_des,
                   channel_descriptors=chn_des)

# prewhitened dataset object
dataset_prewhitened = rsa.data.Dataset(measurements=betas_prewhitened,
                   descriptors=des,
                   obs_descriptors=obs_des,
                   channel_descriptors=chn_des)

Now we calculate the euclidean distances between prewhitened betas using the calc_rdm function in the rsatoolbox.

In [44]:
# calculate euclidean distance between conditions
dist_eucl = rsa.rdm.calc_rdm(dataset_prewhitened, method='euclidean', descriptor='conds')
print(dist_eucl.get_matrices())

[[[0.         1.08975407 1.17226038 0.92475594 1.15762617]
  [1.08975407 0.         1.06488425 0.93402951 0.93152588]
  [1.17226038 1.06488425 0.         1.15108832 1.38364892]
  [0.92475594 0.93402951 1.15108832 0.         0.99174954]
  [1.15762617 0.93152588 1.38364892 0.99174954 0.        ]]]


Then, we calculate the mahalanobis distance between non-prewhitened betas using the inverse of residual mean square in the noise argument. To do this, we need first to diagonalize the residual mean squares vector into a matrix.

In [52]:
# diagonalize ResMS
cov = np.diag(ResMS)

# calculate mahalanobis distance between conditions
dataset = rsa.data.Dataset(measurements=betas,
                   descriptors=des,
                   obs_descriptors=obs_des,
                   channel_descriptors=chn_des)
dist_maha = rsa.rdm.calc_rdm(dataset, method='mahalanobis', descriptor='conds', noise=np.linalg.inv(cov))
print(dist_maha.get_matrices())

AttributeError: measurements must be in dimension n_obs x n_channel

The two representational similarity matrices are numerically identical as expected.

## References

Walther, A., Nili, H., Ejaz, N., Alink, A., Kriegeskorte, N., & Diedrichsen, J. (2016). *Reliability of dissimilarity measures for multi-voxel pattern analysis.* NeuroImage, 137, 188–200. 